## Custom netCDF using goes2go
This was an attempt to solve netCDF processing times - abandoned for remote loading

In [1]:
reset -fs

In [2]:
import xarray as xr 
import s3fs
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from glob import glob

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from rgb import *

import warnings
warnings.filterwarnings('ignore')

import skimage.io as skio
import plotly.io as pio
pio.renderers.default = 'svg'



### Preprocessing reference here:
https://nbviewer.org/github/lsterzinger/goes-l2-latlon-tutorial/blob/main/tutorial-with-outputs.ipynb

In [3]:
# Preprocessing to calculate lat/lon and add to dataset as coordinates
# The math for this function was taken from 
# https://makersportal.com/blog/2018/11/25/goes-r-satellite-latitude-and-longitude-grid-projection-algorithm 


def calc_latlon(ds):

    x = ds.x
    y = ds.y
    goes_imager_projection = ds.goes_imager_projection

    x,y = np.meshgrid(x,y)

    r_eq = goes_imager_projection.attrs["semi_major_axis"]
    r_pol = goes_imager_projection.attrs["semi_minor_axis"]
    l_0 = goes_imager_projection.attrs["longitude_of_projection_origin"] * (np.pi/180)
    h_sat = goes_imager_projection.attrs["perspective_point_height"]
    H = r_eq + h_sat

    a = np.sin(x)**2 + (np.cos(x)**2 * (np.cos(y)**2 + (r_eq**2 / r_pol**2) * np.sin(y)**2))
    b = -2 * H * np.cos(x) * np.cos(y)
    c = H**2 - r_eq**2

    r_s = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)

    s_x = r_s * np.cos(x) * np.cos(y)
    s_y = -r_s * np.sin(x)
    s_z = r_s * np.cos(x) * np.sin(y)

    lat = np.arctan((r_eq**2 / r_pol**2) * (s_z / np.sqrt((H-s_x)**2 +s_y**2))) * (180/np.pi)
    lon = (l_0 - np.arctan(s_y / (H-s_x))) * (180/np.pi)

    ds = ds.assign_coords({
        "lat":(["y","x"],lat),
        "lon":(["y","x"],lon)
    })
    ds.lat.attrs["units"] = "degrees_north"
    ds.lon.attrs["units"] = "degrees_east"

    return ds

# Define function to get x/y bounds given lat/lon bounds
def get_xy_from_latlon(ds, lats, lons):
    lat1, lat2 = lats
    lon1, lon2 = lons

    lat = ds.lat.data
    lon = ds.lon.data

    x = ds.x.data
    y = ds.y.data

    x,y = np.meshgrid(x,y)

    x = x[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)]
    y = y[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)] 

    return ((min(x), max(x)), (min(y), max(y)))

In [4]:
fileset = glob('aws_data/ida_hourly/*.nc')

# # open file with s3fs and xarray
# fs = s3fs.S3FileSystem(anon=True)
# f = fs.open("s3://noaa-goes16/ABI-L2-MCMIPM/2021/241/14/OR_ABI-L2-MCMIPM1-M6_G16_s20212411400278_e20212411400347_c20212411400421.nc")
for f in fileset:
    ds = xr.open_dataset(f)

    
    # Add lat/lon to dataset
    ds = calc_latlon(ds)

    products = ['TrueColor', 'DayCloudConvection', 'DayCloudPhase','WaterVapor']

    for product in products:
        RGB = getattr(ds.rgb, product)()

#         a = px.imshow(RGB)
#         a.show()

    rgb = ds[['TrueColor','DayCloudConvection','DayCloudPhase','WaterVapor']]
#    rgb = ds[['TrueColor']].reset_index(['x', 'y'], drop=True)
    thin = rgb.drop(['y_image', 'x_image','lat', 'lon'])
    thin.to_netcdf(f'netcdf/ida/goes2go/ida_rgb_{rgb.t.dt.strftime("%Y-%d-%m_UTC_%H%M").item()}.nc')
#     rgb.TrueColor.to_netcdf(f'netcdf/ida_tc_{rgb.t.dt.strftime("%Y-%d-%m_UTC_%H%M").item()}.nc')

In [5]:
test = xr.open_dataset('rgb_test.nc')
test
px.imshow(test.DayCloudPhase)

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/noether/Metis/projects/Engineering/rgb_test.nc'